split1

In [1]:
split = 'split1'
path = "../balanceing/balance_data/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

print(train_path)
print(test_path)
print(val_path)

../balanceing/balance_data//split1/train/train_split1.csv
../balanceing/balance_data//test.csv
../balanceing/balance_data//split1/val/val_split1.csv


In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)
print('\n')

print("Shape of training set:", x_train.shape)
print("Shape of testing set:", x_test.shape)

(576, 32)
(576, 30)
['B' 'M']


(144, 32)
(144, 29)
[ 8.571  8.671  8.726  8.734  8.878  9.173  9.405  9.667  9.72   9.777
  9.787 10.03  10.05  10.51  10.57  10.65  10.86  11.06  11.08  11.13
 11.2   11.22  11.27  11.3   11.42  11.43  11.5   11.51  11.6   11.61
 11.64  11.69  11.71  11.75  11.8   11.85  11.87  11.9   11.94  12.05
 12.25  12.27  12.34  12.45  12.46  12.58  12.65  12.77  12.8   12.89
 12.94  12.95  13.    13.03  13.05  13.17  13.24  13.27  13.28  13.34
 13.4   13.46  13.47  13.53  13.61  13.82  13.86  13.87  13.9   14.06
 14.2   14.22  14.25  14.34  14.4   14.47  14.48  14.54  14.71  14.78
 14.86  14.97  15.1   15.3   15.46  15.61  15.66  15.7   15.71  15.73
 15.75  16.11  16.13  16.16  16.24  16.35  16.84  17.01  17.06  17.14
 17.19  17.2   17.3   17.93  18.03  18.08  18.22  18.45  18.81  19.02
 19.1   19.19  19.21  19.27  19.59  19.68  19.73  19.79  20.09  20.13
 20.18  20.26  20.34  20.51  20.58  20.6   20.64  20.73  20.94  21.09
 21.56  21.75  23.27  24.25  25.22 ]


In [3]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

model = SVC(kernel="rbf")
model.fit(x_train, y_train)
predictions = model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM Model is: %.4f" % (acc * 100))

pre = precision_score(y_test, predictions)
print("Precision of SVM Model is: %.4f" % (pre * 100))

sen = recall_score(y_test, predictions)
print("Sensitivity (Recall) of SVM Model is: %.4f" % (sen * 100))

spe = recall_score(y_test, predictions, pos_label=0)
print("Specificity of SVM Model is: %.4f" % (spe * 100))

f1 = f1_score(y_test, predictions)
print("F-score of SVM Model is: %.4f" % (f1 * 100))

ValueError: X has 29 features, but SVC is expecting 30 features as input.

split2

In [ ]:
split = 'split2'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

model = SVC(kernel="rbf")
model.fit(x_train, y_train)
predictions = model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM Model is: %.4f" % (acc * 100))

pre = precision_score(y_test, predictions)
print("Precision of SVM Model is: %.4f" % (pre * 100))

sen = recall_score(y_test, predictions)
print("Sensitivity (Recall) of SVM Model is: %.4f" % (sen * 100))

spe = recall_score(y_test, predictions, pos_label=0)
print("Specificity of SVM Model is: %.4f" % (spe * 100))

f1 = f1_score(y_test, predictions)
print("F-score of SVM Model is: %.4f" % (f1 * 100))

Confusion Matrix: 
 [[46  0]
 [23 45]]


              precision    recall  f1-score   support

           0       0.67      1.00      0.80        46
           1       1.00      0.66      0.80        68

    accuracy                           0.80       114
   macro avg       0.83      0.83      0.80       114
weighted avg       0.87      0.80      0.80       114

Accuracy of SVM Model is: 79.8246
Precision of SVM Model is: 100.0000
Sensitivity (Recall) of SVM Model is: 66.1765
Specificity of SVM Model is: 100.0000
F-score of SVM Model is: 79.6460


split3

In [ ]:
split = 'split3'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

model = SVC(kernel="rbf")
model.fit(x_train, y_train)
predictions = model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM Model is: %.4f" % (acc * 100))

pre = precision_score(y_test, predictions)
print("Precision of SVM Model is: %.4f" % (pre * 100))

sen = recall_score(y_test, predictions)
print("Sensitivity (Recall) of SVM Model is: %.4f" % (sen * 100))

spe = recall_score(y_test, predictions, pos_label=0)
print("Specificity of SVM Model is: %.4f" % (spe * 100))

f1 = f1_score(y_test, predictions)
print("F-score of SVM Model is: %.4f" % (f1 * 100))

Confusion Matrix: 
 [[45  1]
 [20 48]]


              precision    recall  f1-score   support

           0       0.69      0.98      0.81        46
           1       0.98      0.71      0.82        68

    accuracy                           0.82       114
   macro avg       0.84      0.84      0.82       114
weighted avg       0.86      0.82      0.82       114

Accuracy of SVM Model is: 81.5789
Precision of SVM Model is: 97.9592
Sensitivity (Recall) of SVM Model is: 70.5882
Specificity of SVM Model is: 97.8261
F-score of SVM Model is: 82.0513


split4

In [ ]:
split = 'split4'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

model = SVC(kernel="rbf")
model.fit(x_train, y_train)
predictions = model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM Model is: %.4f" % (acc * 100))

pre = precision_score(y_test, predictions)
print("Precision of SVM Model is: %.4f" % (pre * 100))

sen = recall_score(y_test, predictions)
print("Sensitivity (Recall) of SVM Model is: %.4f" % (sen * 100))

spe = recall_score(y_test, predictions, pos_label=0)
print("Specificity of SVM Model is: %.4f" % (spe * 100))

f1 = f1_score(y_test, predictions)
print("F-score of SVM Model is: %.4f" % (f1 * 100))

Confusion Matrix: 
 [[46  0]
 [20 48]]


              precision    recall  f1-score   support

           0       0.70      1.00      0.82        46
           1       1.00      0.71      0.83        68

    accuracy                           0.82       114
   macro avg       0.85      0.85      0.82       114
weighted avg       0.88      0.82      0.83       114

Accuracy of SVM Model is: 82.4561
Precision of SVM Model is: 100.0000
Sensitivity (Recall) of SVM Model is: 70.5882
Specificity of SVM Model is: 100.0000
F-score of SVM Model is: 82.7586


split5

In [ ]:
split = 'split5'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

model = SVC(kernel="rbf")
model.fit(x_train, y_train)
predictions = model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM Model is: %.4f" % (acc * 100))

pre = precision_score(y_test, predictions)
print("Precision of SVM Model is: %.4f" % (pre * 100))

sen = recall_score(y_test, predictions)
print("Sensitivity (Recall) of SVM Model is: %.4f" % (sen * 100))

spe = recall_score(y_test, predictions, pos_label=0)
print("Specificity of SVM Model is: %.4f" % (spe * 100))

f1 = f1_score(y_test, predictions)
print("F-score of SVM Model is: %.4f" % (f1 * 100))

Confusion Matrix: 
 [[46  0]
 [17 51]]


              precision    recall  f1-score   support

           0       0.73      1.00      0.84        46
           1       1.00      0.75      0.86        68

    accuracy                           0.85       114
   macro avg       0.87      0.88      0.85       114
weighted avg       0.89      0.85      0.85       114

Accuracy of SVM Model is: 85.0877
Precision of SVM Model is: 100.0000
Sensitivity (Recall) of SVM Model is: 75.0000
Specificity of SVM Model is: 100.0000
F-score of SVM Model is: 85.7143
